In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import cv2
import numpy as np 
from tqdm import tqdm
import pandas as pd


In [2]:
folder_path = '/kaggle/input/faces-lm/frames'  # Replace with your folder path
folders=sorted(os.listdir(folder_path))

In [3]:
paths=[]
for i in tqdm(range(len(folders))):
    files=os.listdir(os.path.join(folder_path,folders[i]))
    files.sort(key=lambda x: int(x.split('.')[0]))
    files=[os.path.join(folder_path,folders[i],files[j]) for j in range(len(files))]
    paths=paths+files

100%|██████████| 27/27 [00:17<00:00,  1.55it/s]


In [4]:
def get_csv_id(file_path):

    # Extract the filename without extension
    filename = os.path.basename(file_path)  # RightVideoSN001.csv

    # Extract the identifier ('1' or '11')
    if filename.startswith('RightVideoSN'):
        identifier = filename[len('RightVideoSN'):]  # Get '001' or '011'
        identifier = identifier.lstrip('0').split('.')[0]  # Remove leading zeros

    return int(identifier)
def get_id(path):

    # Split the path by '/'
    parts = path.split('/')

    # Get the directory name that contains the image
    dirname = parts[-2]  # RightVideoSN011

    # Extract the relevant parts from the directory name
    if dirname.startswith('RightVideoSN'):
        identifier = dirname[len('RightVideoSN'):]  # Get SN011 -> 011
        identifier = identifier.lstrip('0')  # Remove leading zeros

    # Get the image number from the filename
    filename = parts[-1]  # 240.jpg
    image_number = os.path.splitext(filename)[0]  # 240

    return int(identifier),int(image_number)

In [5]:
folder_path = '/kaggle/input/labels-csv/labels_csv'
csv_path=[]
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        csv_path.append(file_path)
csv_path=sorted(csv_path)
csv={}
for file in csv_path:
    df=pd.read_csv(file)
    ide=get_csv_id(file)
    csv[ide]=df

In [19]:
labels=[]
for file in tqdm(paths):
    ide,image=get_id(file)
    try:
        label=np.array(csv[ide].iloc[image])
        label[label > 0]=1
        labels.append(label)
    except IndexError as e:
        print(f"IndexError occurred: {ide}")
        print(f"Check ide={ide} and image={image} against csv indices.")
labels=np.array(labels)

100%|██████████| 32723/32723 [00:02<00:00, 13539.07it/s]


In [30]:


# Get indices where all elements in rows are zero
zero_indices = np.where(~labels.any(axis=1))[0]

print("Indices where all elements in rows are zero:")

Indices where all elements in rows are zero:
12154


In [25]:
pd.DataFrame(labels,columns=csv[1].columns).to_csv('labels.csv')

In [7]:
frames=[]
target_size=(224, 224)
for i in tqdm(range(len(paths))):
    img=cv2.imread(paths[i],0)
    resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    rgb_image = cv2.cvtColor(resized_img, cv2.COLOR_GRAY2RGB)
    frames.append(rgb_image)

100%|██████████| 32723/32723 [04:21<00:00, 125.03it/s]


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler
import numpy as np
from torchvision import transforms


In [9]:
class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image,torch.tensor(label, dtype=torch.float32)


In [10]:
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image
    transforms.Resize((224, 224)),  # Resize to 224x224 if needed (based on model input requirements)
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (ImageNet stats)
])


In [17]:
dataset = ImageDataset(frames, labels, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [12]:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(self._make_layer(in_channels + i * growth_rate, growth_rate))

    def _make_layer(self, in_channels, growth_rate):
        layer = nn.Sequential(
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
            nn.BatchNorm2d(growth_rate),
            nn.ReLU(inplace=True)
        )
        return layer

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            new_feature = layer(torch.cat(features, 1))
            features.append(new_feature)
        return torch.cat(features, 1)

In [13]:

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.skip_connection = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip_connection(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

In [14]:
class ResiDenNet(nn.Module):
    def __init__(self, growth_rate=32, num_layers=4, num_classes=12):
        super(ResiDenNet, self).__init__()
        self.initial_conv = nn.Conv2d(3, growth_rate * 2, kernel_size=7, stride=2, padding=3)
        self.initial_bn = nn.BatchNorm2d(growth_rate * 2)
        self.initial_relu = nn.ReLU(inplace=True)
        self.initial_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.dense_block1 = DenseBlock(growth_rate * 2, growth_rate, num_layers)
        self.trans1 = self._transition_layer(growth_rate * (2 + num_layers), growth_rate * 2)

        self.res_block = ResidualBlock(growth_rate * 2, growth_rate * 2)
        
        self.dense_block2 = DenseBlock(growth_rate * 2, growth_rate, num_layers)
        self.trans2 = self._transition_layer(growth_rate * (2 + num_layers), growth_rate * 2)

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(growth_rate * 2, num_classes)
        self.sigmoid = nn.Sigmoid()

    def _transition_layer(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = self.initial_relu(x)
        x = self.initial_pool(x)

        x = self.dense_block1(x)
        x = self.trans1(x)
        
        x = self.res_block(x)
        
        x = self.dense_block2(x)
        x = self.trans2(x)

        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResiDenNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [18]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

Epoch 1, Loss: 0.7856090814057142
Validation Loss: 0.7214972373915882
Epoch 2, Loss: 0.7083842315516629
Validation Loss: 0.7016124766047408
Epoch 3, Loss: 0.697740518784785
Validation Loss: 0.6966684614739767
Epoch 4, Loss: 0.695117383722275
Validation Loss: 0.6944540384339123
Epoch 5, Loss: 0.694134972325466
Validation Loss: 0.6938022427442597
Epoch 6, Loss: 0.6935142323793217
Validation Loss: 0.692061056741854
Epoch 7, Loss: 0.6881064255886753
Validation Loss: 0.6854361917914413
Epoch 8, Loss: 0.6850937698786949
Validation Loss: 0.6854333807782429
Epoch 9, Loss: 0.6844148592197852
Validation Loss: 0.6839474032564861
Epoch 10, Loss: 0.6842028278134245
Validation Loss: 0.6837929463968044
Epoch 11, Loss: 0.6839261607372717
Validation Loss: 0.683530414395216
Epoch 12, Loss: 0.6836947245621128
Validation Loss: 0.6842107525685939
Epoch 13, Loss: 0.683571392290467
Validation Loss: 0.6834773534681738
Epoch 14, Loss: 0.6830260255106785
Validation Loss: 0.6818618274316555
Epoch 15, Loss: 0.680

In [51]:
import numpy as np

model.eval()  # Set model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        
        # Apply sigmoid activation to outputs to get probabilities
        probabilities = torch.sigmoid(outputs)
        
        # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
        threshold = 0.5
        predicted_labels = (probabilities > threshold).int()
        
        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())  # Optional: Collect true labels for evaluation

# Now predictions contains the predicted labels (binary) for your test dataset
predictions = np.array(predictions)
print(predictions)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, concatenate
from tensorflow.keras.models import Model

# Define the ResiDen model
def ResiDen(input_shape):
    inputs = Input(shape=input_shape)

    # Residual Dense Network Layers
    x = Conv2D(48, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    residen_features = Dense(4096, activation='relu')(x)
    
    model = Model(inputs, residen_features)
    return model

# Define the basic CNN model
def BasicCNN(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(48, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    cnn_features = Dense(2048, activation='relu')(x)
    
    model = Model(inputs, cnn_features)
    return model

# Define the combined model
def CombinedModel(input_shape):
    # ResiDen model
    residen_model = ResiDen(input_shape)
    residen_input = residen_model.input
    residen_output = residen_model.output

    # Basic CNN model
    cnn_model = BasicCNN(input_shape)
    cnn_input = cnn_model.input
    cnn_output = cnn_model.output

    # Concatenate features from both models
    combined_features = concatenate([residen_output, cnn_output])

    # Final fully connected layers
    x = Dense(512, activation='relu')(combined_features)
    x = Dense(2048, activation='relu')(x)
    x = Dense(2048, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)  # Assuming a classification task

    model = Model(inputs=[residen_input, cnn_input], outputs=outputs)
    return model

# Define input shape and number of classes
input_shape = (64, 64, 3)  # Example input shape, adjust as needed
num_classes = 7  # Example number of classes, adjust as needed

# Create the combined model
model = CombinedModel(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()
